# Brent Oil Price Change Point Analysis

This notebook performs Bayesian change point detection on Brent oil prices to identify structural breaks and associate them with major geopolitical and economic events.

## 1. Data Loading and Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

In [ ]:
# Load Brent oil price data
df_prices = pd.read_csv('../data/brent_prices.csv')

# Convert Date column to datetime
# Handle the format '20-May-87'
df_prices['Date'] = pd.to_datetime(df_prices['Date'], format='%d-%b-%y')

# Sort by date
df_prices = df_prices.sort_values('Date').reset_index(drop=True)

print(f"Data shape: {df_prices.shape}")
print(f"Date range: {df_prices['Date'].min()} to {df_prices['Date'].max()}")
print(f"Price range: ${df_prices['Price'].min():.2f} to ${df_prices['Price'].max():.2f}")
df_prices.head()

In [ ]:
# Load events data
df_events = pd.read_csv('../data/events.csv')
df_events['Date'] = pd.to_datetime(df_events['Date'])
df_events = df_events.sort_values('Date').reset_index(drop=True)

print(f"Number of events: {len(df_events)}")
df_events.head(10)

## 2. Exploratory Data Analysis

In [ ]:
# Plot raw price series
plt.figure(figsize=(16, 6))
plt.plot(df_prices['Date'], df_prices['Price'], linewidth=0.8, alpha=0.7)
plt.title('Brent Oil Price Time Series (1987-2022)', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (USD per barrel)', fontsize=12)
plt.grid(True, alpha=0.3)

# Add event markers
for _, event in df_events.iterrows():
    plt.axvline(x=event['Date'], color='red', linestyle='--', alpha=0.5, linewidth=1)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate log returns for stationarity analysis
df_prices['Log_Price'] = np.log(df_prices['Price'])
df_prices['Log_Returns'] = df_prices['Log_Price'].diff()
df_prices = df_prices.dropna().reset_index(drop=True)

# Plot log returns
plt.figure(figsize=(16, 6))
plt.plot(df_prices['Date'], df_prices['Log_Returns'], linewidth=0.5, alpha=0.7)
plt.title('Brent Oil Log Returns (Stationarity Analysis)', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Log Returns', fontsize=12)
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Stationarity test
from statsmodels.tsa.stattools import adfuller

# Test on prices
result_prices = adfuller(df_prices['Price'].dropna())
print("ADF Test on Prices:")
print(f"ADF Statistic: {result_prices[0]:.4f}")
print(f"p-value: {result_prices[1]:.4f}")
print(f"Is Stationary: {result_prices[1] < 0.05}")
print()

# Test on log returns
result_returns = adfuller(df_prices['Log_Returns'].dropna())
print("ADF Test on Log Returns:")
print(f"ADF Statistic: {result_returns[0]:.4f}")
print(f"p-value: {result_returns[1]:.4f}")
print(f"Is Stationary: {result_returns[1] < 0.05}")

In [ ]:
# Rolling volatility
window = 30  # 30-day rolling window
df_prices['Rolling_Volatility'] = df_prices['Log_Returns'].rolling(window=window).std() * np.sqrt(252)

plt.figure(figsize=(16, 6))
plt.plot(df_prices['Date'], df_prices['Rolling_Volatility'], linewidth=1)
plt.title(f'Rolling Volatility ({window}-day window, annualized)', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Volatility', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Bayesian Change Point Model with PyMC

In [ ]:
import pymc as pm
import arviz as az

# Prepare data for modeling
# We'll model the price levels (not returns) to detect mean shifts
prices = df_prices['Price'].values
n = len(prices)
time_index = np.arange(n)

print(f"Number of observations: {n}")
print(f"Price mean: ${prices.mean():.2f}")
print(f"Price std: ${prices.std():.2f}")

In [ ]:
# Build the Bayesian Change Point Model
with pm.Model() as change_point_model:
    # Switch point (tau) - discrete uniform prior over all possible days
    tau = pm.DiscreteUniform("tau", lower=1, upper=n-1)
    
    # Before and after mean parameters
    # Use informative priors based on data
    mu_before = pm.Normal("mu_before", mu=prices.mean(), sigma=prices.std())
    mu_after = pm.Normal("mu_after", mu=prices.mean(), sigma=prices.std())
    
    # Standard deviation (assumed constant for simplicity)
    sigma = pm.HalfNormal("sigma", sigma=prices.std())
    
    # Switch function to select the correct mean based on tau
    mu = pm.math.switch(time_index < tau, mu_before, mu_after)
    
    # Likelihood - Normal distribution
    likelihood = pm.Normal("likelihood", mu=mu, sigma=sigma, observed=prices)
    
    # Sample from posterior
    trace = pm.sample(2000, tune=1000, return_inferencedata=True, random_seed=42)

print("Sampling complete!")

## 4. Model Diagnostics and Convergence

In [ ]:
# Check convergence
summary = az.summary(trace)
print("Model Summary:")
print(summary)
print("\nR-hat values close to 1.0 indicate good convergence.")

In [ ]:
# Plot trace plots
az.plot_trace(trace, var_names=['tau', 'mu_before', 'mu_after', 'sigma'])
plt.tight_layout()
plt.show()

## 5. Change Point Identification

In [ ]:
# Extract posterior distribution of tau
tau_samples = trace.posterior['tau'].values.flatten()

# Plot posterior distribution of change point
plt.figure(figsize=(14, 6))
plt.hist(tau_samples, bins=50, alpha=0.7, edgecolor='black')
plt.axvline(tau_samples.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {int(tau_samples.mean())}')
plt.axvline(np.median(tau_samples), color='green', linestyle='--', linewidth=2, label=f'Median: {int(np.median(tau_samples))}')
plt.xlabel('Change Point Index (tau)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Posterior Distribution of Change Point (tau)', fontsize=16, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Convert to dates
most_likely_tau = int(np.median(tau_samples))
change_point_date = df_prices.iloc[most_likely_tau]['Date']
print(f"Most likely change point index: {most_likely_tau}")
print(f"Most likely change point date: {change_point_date.strftime('%Y-%m-%d')}")

In [ ]:
# Plot change point on price series
plt.figure(figsize=(16, 6))
plt.plot(df_prices['Date'], df_prices['Price'], linewidth=0.8, alpha=0.7, label='Brent Oil Price')
plt.axvline(x=change_point_date, color='red', linestyle='--', linewidth=2, label=f'Detected Change Point: {change_point_date.strftime("%Y-%m-%d")}')

# Add event markers
for _, event in df_events.iterrows():
    plt.axvline(x=event['Date'], color='orange', linestyle=':', alpha=0.5, linewidth=1)

plt.title('Brent Oil Prices with Detected Change Point', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (USD per barrel)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Impact Quantification

In [ ]:
# Extract posterior distributions of before/after parameters
mu_before_samples = trace.posterior['mu_before'].values.flatten()
mu_after_samples = trace.posterior['mu_after'].values.flatten()
sigma_samples = trace.posterior['sigma'].values.flatten()

# Plot posterior distributions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].hist(mu_before_samples, bins=50, alpha=0.7, edgecolor='black', color='blue')
axes[0].axvline(mu_before_samples.mean(), color='red', linestyle='--', linewidth=2)
axes[0].set_xlabel('Mean Before Change Point (USD)', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title(f'Posterior: μ₁ (Before)\nMean: ${mu_before_samples.mean():.2f}', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

axes[1].hist(mu_after_samples, bins=50, alpha=0.7, edgecolor='black', color='green')
axes[1].axvline(mu_after_samples.mean(), color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Mean After Change Point (USD)', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
axes[1].set_title(f'Posterior: μ₂ (After)\nMean: ${mu_after_samples.mean():.2f}', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

axes[2].hist(sigma_samples, bins=50, alpha=0.7, edgecolor='black', color='purple')
axes[2].axvline(sigma_samples.mean(), color='red', linestyle='--', linewidth=2)
axes[2].set_xlabel('Standard Deviation (USD)', fontsize=11)
axes[2].set_ylabel('Frequency', fontsize=11)
axes[2].set_title(f'Posterior: σ\nMean: ${sigma_samples.mean():.2f}', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate impact metrics
mean_before = mu_before_samples.mean()
mean_after = mu_after_samples.mean()
absolute_change = mean_after - mean_before
percentage_change = (absolute_change / mean_before) * 100

# Calculate credible intervals (95%)
ci_before = np.percentile(mu_before_samples, [2.5, 97.5])
ci_after = np.percentile(mu_after_samples, [2.5, 97.5])

print("=" * 60)
print("IMPACT QUANTIFICATION")
print("=" * 60)
print(f"\nChange Point Date: {change_point_date.strftime('%Y-%m-%d')}")
print(f"\nBefore Change Point:")
print(f"  Mean Price: ${mean_before:.2f}")
print(f"  95% Credible Interval: [${ci_before[0]:.2f}, ${ci_before[1]:.2f}]")
print(f"\nAfter Change Point:")
print(f"  Mean Price: ${mean_after:.2f}")
print(f"  95% Credible Interval: [${ci_after[0]:.2f}, ${ci_after[1]:.2f}]")
print(f"\nImpact:")
print(f"  Absolute Change: ${absolute_change:.2f}")
print(f"  Percentage Change: {percentage_change:.2f}%")
print("=" * 60)

## 7. Event Association

In [ ]:
# Find events near the change point
time_window_days = 90  # Look for events within 90 days

change_point_datetime = pd.to_datetime(change_point_date)

# Calculate time differences
df_events['Days_Diff'] = (df_events['Date'] - change_point_datetime).dt.days.abs()
nearby_events = df_events[df_events['Days_Diff'] <= time_window_days].sort_values('Days_Diff')

print(f"Events within {time_window_days} days of detected change point:")
print("=" * 80)
if len(nearby_events) > 0:
    for _, event in nearby_events.iterrows():
        print(f"\nEvent: {event['Event']}")
        print(f"Date: {event['Date'].strftime('%Y-%m-%d')}")
        print(f"Days from change point: {int(event['Days_Diff'])}")
        print(f"Category: {event['Category']}")
        print(f"Impact Level: {event['Impact_Level']}")
        print(f"Description: {event['Description']}")
        print("-" * 80)
else:
    print("No events found within the specified time window.")

## 8. Summary and Interpretation

### Key Findings:

1. **Change Point Detection**: The Bayesian model identified a structural break in Brent oil prices.

2. **Impact Quantification**: The model estimates the price level shifted from approximately $X to $Y, representing a Z% change.

3. **Event Association**: The detected change point is temporally associated with [specific events], suggesting these events may have contributed to the structural break.

4. **Uncertainty**: The posterior distributions provide probabilistic estimates with credible intervals, allowing for uncertainty quantification.

### Limitations:

- This analysis identifies **correlation**, not **causation**
- The model assumes a single change point; multiple breaks may exist
- External factors not included in the event dataset may also contribute
- The timing of market impacts may differ from event dates (anticipation, delays)